<a href="https://colab.research.google.com/github/langroid/groq-api-cookbook/blob/main/langroid-llm-agents/search-assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Get your `GROQ_API_KEY` here:

https://console.groq.com/keys



In [1]:
!pip install -q --upgrade langroid &> /dev/null
!pip show langroid


Name: langroid
Version: 0.1.239
Summary: Harness LLMs with Multi-Agent Programming
Home-page: 
Author: Prasad Chalasani
Author-email: pchalasani@gmail.com
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-generator, autopep8, black, bs4, colorlog, docstring-parser, duckduckgo-search, faker, fakeredis, fire, flake8, google-api-python-client, groq, grpcio, halo, jinja2, lancedb, lxml, meilisearch, meilisearch-python-sdk, momento, mypy, nest-asyncio, nltk, onnxruntime, openai, pandas, pdf2image, pdfplumber, pre-commit, prettytable, pyarrow, pydantic, pygithub, pygments, pymupdf, pyparsing, pypdf, pytesseract, pytest-asyncio, pytest-redis, python-docx, python-dotenv, python-magic, qdrant-client, rank-bm25, redis, requests, requests-oauthlib, rich, ruff, scrapy, sqlalchemy, tantivy, thefuzz, tiktoken, trafilatura, typer, types-pillow, types-pyyaml, types-redis, types-requests, wget
Required-by: 


In [9]:
import os
import getpass
os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your GROQ_API_KEY: ")

Enter your GROQ_API_KEY: ··········


In [10]:
import langroid as lr
import langroid.language_models as lm
from langroid.agent.tools.duckduckgo_search_tool import DuckduckgoSearchTool
from langroid.utils.configuration import settings
settings.notebook = True

MODEL = "groq/llama3-70b-8192"


Configure the LLM, define the Assistant Agent and Task

In [11]:
llm_config = lm.OpenAIGPTConfig(
      chat_model=MODEL, # or lr.OpenAIChatModel.GPT4_TURBO
      chat_context_length=8192,
      temperature=0,
      max_output_tokens=200,
      timeout=45,
  )

assistant_config = lr.ChatAgentConfig(
    system_message="""
    You are a resourceful assistant, able to think step by step to answer
    complex questions from the user. You must break down complex questions into
    simpler questions that can be answered by a web search. You must ask me
    (the user) each question ONE BY ONE, and I will do a web search and send you
    a brief answer. Once you have enough information to answer my original
    (complex) question, you MUST say DONE and present the answer to me.
    """,
    llm=llm_config,
    vecdb=None,
)
assistant_agent = lr.ChatAgent(assistant_config)

assistant_task = lr.Task(
    assistant_agent,
    name="Assistant",
    llm_delegate=True,
    single_round=False,
    interactive=False,
)





                    using fake redis client
2024-05-03 12:07:20 - WARNING - REDIS_PASSWORD, REDIS_HOST, REDIS_PORT not set in .env file,
                    using fake redis client


Define the Searcher Agent and Task


In [12]:
search_tool_handler_method = DuckduckgoSearchTool.default_value("request")

search_agent_config = lr.ChatAgentConfig(
    llm=llm_config,
    vecdb=None,
    system_message=f"""
    You are a web-searcher. For any question you get, you must use the
    `{search_tool_handler_method}` tool/function-call to get up to 5 results.
    I WILL SEND YOU THE RESULTS; DO NOT MAKE UP THE RESULTS!!
    Once you receive the results, you must compose a CONCISE answer
    based on the search results and say DONE and show the answer to me,
    in this format:
    DONE [... your CONCISE answer here ...]
    IMPORTANT: YOU MUST WAIT FOR ME TO SEND YOU THE
    SEARCH RESULTS BEFORE saying you're DONE.
    """,
)
search_agent = lr.ChatAgent(search_agent_config)
search_agent.enable_message(DuckduckgoSearchTool)


search_task = lr.Task(
    search_agent,
    name="Searcher",
    llm_delegate=True,
    single_round=False,
    interactive=False,
)

                    using fake redis client
2024-05-03 12:07:23 - WARNING - REDIS_PASSWORD, REDIS_HOST, REDIS_PORT not set in .env file,
                    using fake redis client


Make the Searcher Task a sub-task of the Assistant Task, and run the Assistant Task.

In [13]:
assistant_task.add_sub_task(search_task)
question = "Was List born before Wagner?"
assistant_task.run(question)

>>> Starting Agent Assistant (1) groq/llama3-70b-8192 

Let's break this down step by step.

Here's my first question:

When was Franz Liszt born?

Stats: N_MSG=2, TOKENS: in=108, out=22, max=200, ctx=8192, COST: now=$0.0000, cumul=$0.0000 (llama3-70b-8192)

...|>>> Starting Agent Searcher (1) groq/llama3-70b-8192 

...|

...

Now that we have both birthdates, I can answer the original question.

DONE!

Franz Liszt was born before Richard Wagner.

Stats: N_MSG=6, TOKENS: in=190, out=27, max=200, ctx=8192, COST: now=$0.0000, cumul=$0.0000 (llama3-70b-8192)

Bye, hope this was useful!

<<< Finished Agent Assistant (7) 

ChatDocument(content='Now that we have both birthdates, I can answer the original question.\n\n!\n\nFranz Liszt was born before Richard Wagner.', metadata=ChatDocMetaData(source='User', is_chunk=False, id='', window_ids=[], parent=None, sender=<Entity.USER: 'User'>, tool_ids=[], parent_responder=None, block=None, sender_name='Assistant', recipient='', usage=None, cached=False, displayed=False, status=<StatusCode.DONE: 'DONE'>), function_call=None, tool_messages=[], attachment=None)